<a href="https://colab.research.google.com/github/dlwltn0430/autonomous-driving-samples/blob/main/traffic_light.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# 데이터 경로 설정
train_dir = 'dataset/train'
validation_dir = 'dataset/validation'

# 이미지 데이터 제너레이터 설정
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# 데이터 제너레이터 생성
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# 모델 생성
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

# 모델 저장
model.save('traffic_light_model.h5')

Found 80 images belonging to 3 classes.
Found 43 images belonging to 3 classes.
Epoch 1/25
2/2 [==============================] - 9s 5s/step - loss: 0.8596 - accuracy: 0.5208 - val_loss: 0.8504 - val_accuracy: 0.4375
Epoch 2/25
2/2 [==============================] - 7s 5s/step - loss: 0.7415 - accuracy: 0.5417 - val_loss: 0.7240 - val_accuracy: 0.4688
Epoch 3/25
2/2 [==============================] - 6s 4s/step - loss: 0.6906 - accuracy: 0.5417 - val_loss: 0.6355 - val_accuracy: 1.0000
Epoch 4/25
2/2 [==============================] - 7s 4s/step - loss: 0.6767 - accuracy: 0.6719 - val_loss: 0.6136 - val_accuracy: 0.5312
Epoch 5/25
2/2 [==============================] - 5s 2s/step - loss: 0.6883 - accuracy: 0.6250 - val_loss: 0.5272 - val_accuracy: 1.0000
Epoch 6/25
2/2 [==============================] - 8s 6s/step - loss: 0.6118 - accuracy: 0.7083 - val_loss: 0.4039 - val_accuracy: 1.0000
Epoch 7/25
2/2 [==============================] - 5s 2s/step - loss: 0.4633 - accuracy: 0.7500 - v

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# 모델 로드
model = load_model('traffic_light_model.h5')

# 클래스 레이블
class_labels = ['Red', 'Yellow', 'Green']

def classify_traffic_light(image_path):
    # 이미지 로드
    image = cv2.imread(image_path)
    image = cv2.resize(image, (150, 150))
    image = img_to_array(image) / 255.0
    image = np.expand_dims(image, axis=0)

    # 예측
    prediction = model.predict(image)
    class_idx = np.argmax(prediction)
    class_label = class_labels[class_idx]

    return class_label

# 예시 이미지 분류
image_path = 'dataset/test/red_light.jpg'
result = classify_traffic_light(image_path)
print(f"The traffic light is: {result}")

image_path2 = 'dataset/test/green_light.jpg'
result2 = classify_traffic_light(image_path2)
print(f"The traffic light is: {result2}")

1/1 [==============================] - 0s 124ms/step
The traffic light is: Yellow
1/1 [==============================] - 0s 40ms/step
The traffic light is: Green
